# Preprocessing customers.csv Part 1

In [8]:
import numpy as np
import pandas as pd
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords

pd.options.display.float_format = '{:.4f}'.format

### Loading Data via S3

In [ ]:
bucket='ads-508-group-6-raw'
data_key = 'articles.csv'

s3 = boto3.client('s3')
articles_obj = s3.get_object(Bucket = bucket, Key = data_key)

articles_df = pd.read_csv(articles_obj['Body'])

### Loading Data Locally

In [2]:
articles_df = pd.read_csv("C:\\Users\\garyb\\OneDrive\\Documents\\GitHub\\recommendation-engine\\data\\articles.csv")

In [3]:
articles_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [6]:
articles_df.describe()

,article_id,product_code,product_type_no,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_group_no,section_no,garment_group_no
count,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000,105542.0000
mean,698424569.0969,698424.5634,234.8619,1009515.0757,32.2338,3.2062,7.8080,4532.7778,3.1715,42.6642,1010.4383
std,128462381.3362,128462.3844,75.0493,22413.5858,28.0862,1.5638,5.3767,2712.6920,4.3532,23.2601,6.7310
min,108775015.0000,108775.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,1201.0000,1.0000,2.0000,1001.0000
25%,616992501.0000,616992.5000,252.0000,1010008.0000,9.0000,2.0000,4.0000,1676.0000,1.0000,20.0000,1005.0000
50%,702213001.5000,702213.0000,259.0000,1010016.0000,14.0000,4.0000,5.0000,4222.0000,2.0000,46.0000,1009.0000
75%,796703001.7500,796703.0000,272.0000,1010016.0000,52.0000,4.0000,11.0000,7389.0000,4.0000,61.0000,1017.0000
max,959461001.0000,959461.0000,762.0000,1010029.0000,93.0000,7.0000,20.0000,9989.0000,26.0000,97.0000,1025.0000


### Recast Variables

In [7]:
articles_df = articles_df.astype({'article_id':'str',
                                  'product_code':'str',
                                  'graphical_appearance_no':'str',
                                  'colour_group_code':'str',
                                  'perceived_colour_value_id':'str',
                                  'perceived_colour_master_id':'str',
                                  'department_no':'str',
                                  'index_group_no':'str',
                                  'section_no':'str',
                                  'garment_group_no':'str'})

### Text Cleaning

In [13]:
def clean_tweet(self, text): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

In [14]:
articles_df['prod_name'] = articles_df['prod_name'].map(clean_text)
articles_df['product_type_name'] = articles_df['product_type_name'].map(clean_text)
articles_df['product_group_name'] = articles_df['product_group_name'].map(clean_text)
articles_df['graphical_appearance_name'] = articles_df['graphical_appearance_name'].map(clean_text)
articles_df['colour_group_name'] = articles_df['colour_group_name'].map(clean_text)
articles_df['perceived_colour_value_name'] = articles_df['perceived_colour_value_name'].map(clean_text)
articles_df['perceived_colour_master_name'] = articles_df['perceived_colour_master_name'].map(clean_text)
articles_df['department_name'] = articles_df['department_name'].map(clean_text)
articles_df['index_name'] = articles_df['index_name'].map(clean_text)
articles_df['index_group_name'] = articles_df['index_group_name'].map(clean_text)
articles_df['section_name'] = articles_df['section_name'].map(clean_text)
articles_df['garment_group_name'] = articles_df['garment_group_name'].map(clean_text)
articles_df['detail_desc'] = articles_df['detail_desc'].map(clean_text)

TypeError: expected string or bytes-like object